<a href="https://colab.research.google.com/github/McLavish/telegramBot/blob/master/ProgettoTipsit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programma WOW per generare testo con GTP-2 VELOCEMENTE :0

In [0]:
#Dependency per il machine learning
!pip install torch torchvision
!pip install transformers

#modelli disponibili: https://huggingface.co/models?search=gpt2-

In [0]:
import random
import torch

from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer
)

class GeneratoreTesto:
    def __init__(self):
      seed = random.randint(0, 2147483647)
      torch.random.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

      #imposto la GPU come dispositivo da utilizzare (La CPU è lenta)
      self.device = "cuda"

      #lista modelli: https://huggingface.co/models?search=gpt2-
      model_name_or_path = "gpt2-xl"

      self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)

      self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

      #imposto modalità evaluation del modello (no training)
      self.model.eval()
      self.model.to(self.device)
      
    def generate_text(self,
      prompt="",
      #lunghezza generazione
      length=20,
      #se devo fermarmi prima a leggere il testo generato
      stop_token = None,
      temperature=1.0,
      repetition_penalty=1.0,
      k=0,
      p=0.9,
      #quanti campioni voglio generare (io 1 alla volta)
      num_return_sequences=1,
    ):
      #Prendo la lunghezza massima del modello
      lunghezza_massima = self.model.config.max_position_embeddings

      print(lunghezza_massima);
      
      #Controllo che la lunghezza richiesta + la lunghezza dell'input siano minori del modello
      if length < 0 or length > lunghezza_massima:
        length = lunghezza_massima

      #Codifico il testo
      encoded_prompt = self.tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
      encoded_prompt = encoded_prompt.to(self.device)

      #Genero la sequenza
      output_sequences = self.model.generate (
          input_ids=encoded_prompt,
          max_length=length + len(encoded_prompt[0]),
          temperature=temperature,
          top_k=k,
          top_p=p,
          repetition_penalty=repetition_penalty,
          do_sample=True,
          num_return_sequences=num_return_sequences,
          )
      
      #Prendo solo il primo batch (potrei generare più batch in parallelo ma al mio programma non serve)
      generated_sequence = output_sequences[0]

      # Decodifico il testo
      text = self.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

      # Rimuovo il prompt iniziale dal testo (perchè ti ridà anche quello dopo che ha generato)
      total_sequence = (text[len(prompt) :])

      # Se è stato specificato uno stop token rimuovo tutto quello che c'è dopo di lui
      text = text[: text.find(stop_token) if stop_token else None]

      return total_sequence


In [0]:
#Istanzio l'oggetto che mi genera il testo e faccio una prova (così carica in memoria il modello)

generator = GeneratoreTesto()

generator.generate_text(
            length=50,
            prompt="""Hamlet was once a man of honor"""
        )

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


1024


', that he never believed in or practised the love of riches. He died rich and happy.\n\nAs I glance upon his remains, my heart bleeds for the smallness of his life. He was only sixty years of age when he'

In [0]:
#Creo un tunnel con ngrok perchè Google Colab non permette la creazione di API esposte

!if [ ! -f "ngrok-stable-linux-amd64.zip" ]; then wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; fi
!if [ ! -f "ngrok" ]; then unzip -o ngrok-stable-linux-amd64.zip; fi

!./ngrok authtoken "1aMSPEPk5WpPiquwebio1IWcILq_2RRMqrTUj968qnmrXss6L"

get_ipython().system_raw('./ngrok http 5000 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://2db4373c.ngrok.io


In [0]:
#Dependency per il web server con FastAPI

!pip install fastapi
!pip install uvicorn
!pip install pydantic

In [0]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

class Richiesta(BaseModel):
    prompt: str
    length: int = None
    stop_token: str = None
    temperature: float = None
    repetition_penalty: float = None
    k: float = None
    p: float = None

app = FastAPI()

@app.get("/")
def root():
      return {"string": "Hello World"}

@app.post("/predict/")
def predict(richiesta: Richiesta):

      output = generator.generate_text (
            prompt = richiesta.prompt,
            length = richiesta.length,
            stop_token = richiesta.stop_token,
            temperature = richiesta.temperature,
            repetition_penalty = richiesta.repetition_penalty,
            k = richiesta.k,
            p = richiesta.p
        )
      
      return {"result": output}

uvicorn.run(app,port=5000)

INFO:     Started server process [1196]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


In [0]:
#Codice vecchio per flask
"""
from flask import Flask, abort, jsonify, request

app = Flask(__name__)

@app.route("/")
def home():
      prompt = request.args.get('prompt')
      output = generate_text(
            model_type='gpt2',
            length=20,
            prompt='hello world',
            model_name_or_path='gpt2-large'
        )
      #abort(404, description="Resource not found")
      return jsonify({'result': output})

@app.route("/predict")
def predict():
    return "no"

app.run();
"""

In [0]:
#QUESTA CELLA è PERICOLOSA! FA CRASHARE COLAB COSì TI DA Più MEMORIA
"""
a = []
while(1):
    a.append('1')
"""

In [0]:
#PER VEDERE CHE SCHEDA GRAFICA MI è STATA ASSEGNATA (Tesla P100 PCIE è la migliore, Tesla K80 è la più schifosa)
!nvidia-smi

In [0]:
#codice per montare il mio google drive
"""
from google.colab import drive
drive.mount('/content/drive')
"""

#Prova quick & dirty del modello
"""
!wget https://raw.githubusercontent.com/huggingface/pytorch-transformers/master/examples/run_generation.py

!python run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path=gpt2
"""

In [0]:
#CODICE PER PREVENIRE IL RUNTIME DAL DISCONNETTERSI PER INATTIVITà:
#ATTENZIONE: Potrebbe causare la revoca dei privilegi di COLAB se lo si lascia andare per troppo tempo!

#function ClickConnect(){console.log("Working");document.querySelector("colab-toolbar-button#connect").click()}setInterval(ClickConnect,60000)
